In [ ]:
# Loading Libarary
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
# real (0), fake (1)
df = pd.read_csv('/content/drive/MyDrive/DataSets/WELFake_Dataset[1].csv')
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [ ]:
df.shape

(72134, 4)

In [ ]:
df.drop_duplicates(inplace=True)
df.shape

(72134, 4)

In [ ]:
df.isnull().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [ ]:
df.dropna(axis=0, inplace=True)
df.shape

(71537, 4)

In [ ]:
# Combine Important column
df['combined'] = df['text']  + ' ' + df['title']
df.head()

,Unnamed: 0,title,text,label,combined
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,No comment is expected from Barack Obama Membe...
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,"Now, most of the demonstrators gathered last ..."
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,A dozen politically active pastors came here f...
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1,All we can say on this one is it s about time ...


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def process_text(text):
  nopunc = [char for char in text if char not in string.punctuation]
  nopunc = ''.join(nopunc)
  clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
  return clean_words

In [ ]:
df['combined'].head().apply(process_text)

0    [comment, expected, Barack, Obama, Members, FY...
2    [demonstrators, gathered, last, night, exercis...
3    [dozen, politically, active, pastors, came, pr...
4    [RS28, Sarmat, missile, dubbed, Satan, 2, repl...
5    [say, one, time, someone, sued, Southern, Pove...
Name: combined, dtype: object

In [ ]:
df['combined']

0        No comment is expected from Barack Obama Membe...
2         Now, most of the demonstrators gathered last ...
3        A dozen politically active pastors came here f...
4        The RS-28 Sarmat missile, dubbed Satan 2, will...
5        All we can say on this one is it s about time ...
                               ...                        
72129    WASHINGTON (Reuters) - Hackers believed to be ...
72130    You know, because in fantasyland Republicans n...
72131    Migrants Refuse To Leave Train At Refugee Camp...
72132    MEXICO CITY (Reuters) - Donald Trump’s combati...
72133    Goldman Sachs Endorses Hillary Clinton For Pre...
Name: combined, Length: 71537, dtype: object

In [24]:
#from sklearn.feature_extraction.text import CountVectorizer
#message_bow = CountVectorizer(analyzer=process_text).fit_transform(df['combined'])

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

# Ensure process_text is defined
def process_text(text):
    # Example processing function
    return text.split()

# Create the CountVectorizer with the custom analyzer
vectorizer = CountVectorizer(analyzer=process_text)

# Fit and transform the data
message_bow = vectorizer.fit_transform(df['combined'])


In [19]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(message_bow, df['label'], test_size=0.2, random_state=0)

In [20]:
message_bow.shape

(71537, 805861)

In [21]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(x_train, y_train)

MultinomialNB()

In [ ]:
print(classifier.predict(x_train))
print(y_train.values)

In [22]:
from sklearn.metrics import classification_report
pred = classifier.predict(x_train)
print(classification_report(y_train, pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97     28065
           1       0.98      0.96      0.97     29164

    accuracy                           0.97     57229
   macro avg       0.97      0.97      0.97     57229
weighted avg       0.97      0.97      0.97     57229



In [23]:
from sklearn.metrics import classification_report
pred = classifier.predict(x_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.94      0.97      0.95      6963
           1       0.97      0.94      0.95      7345

    accuracy                           0.95     14308
   macro avg       0.95      0.95      0.95     14308
weighted avg       0.95      0.95      0.95     14308

